In [1]:
import json
import gzip
from itertools import islice
from collections import Counter, defaultdict
from operator import itemgetter

import pandas as pd
from tqdm.notebook import tqdm

In [20]:
data_path = '..\\Data\\In\\'
submissions_path =  '..\\Submissions\\'

## Чтение входных данных

Напишем простую функцию для чтения файла с частью вакансий:

In [5]:
def read_vacancies_part(part):
    with gzip.open(data_path+f'vacancies-{part:02}.json.gz', 'r') as fp:
        return json.loads(fp.read())

In [22]:
%%time

part = read_vacancies_part(1)
print(f'Всего вакансий в части 1: {len(part):,d}')

Всего вакансий в части 1: 300,000
Wall time: 9.61 s


Пример содержимого вакансии:

In [26]:
part['2']

{'name': 'Супервайзер команды Службы поддержки пользователей',
 'description': '<p>В связи с расширением, в международной компании OLX Group открыта вакансия <strong>Супервайзера команды Службы поддержки пользователей</strong>.</p> <p>Если ты целеустремленный, ты хочешь помогать другим и развиваться вместе с OLX - присоединяйся к нашей большой и дружной команде!</p> <p>Ниже ты можешь ознакомиться с основными обязанностями, требованиями, а также условиями работы на данной позиции. Мы стремимся создать комфортные условия каждому члену нашей команды и заинтересованы в долгосрочном сотрудничестве с достойными кандидатами.</p> <p><strong>Обязанности:</strong></p> <ul> <li>работа с командой;</li> <li>обучение новых сотрудников, адаптация;</li> <li>организация и проведение тренингов, ежемесячных тестирований;</li> <li>наполнение и актуализация базы знаний для агентов, написание инструкций и коммуникация изменений;</li> <li>статистика и аналитика KPI;</li> <li>сбор и передача данных по агентам

In [7]:
print(json.dumps(part['20'], indent=4, ensure_ascii=False))

{
    "name": "Менеджер по оптовой продаже медицинских оправ и солнцезащитных очков",
    "description": "<strong>Обязанности:</strong> <ul> <li> <p>- оптовые продажи медицинских оправ и солнцезащитных очков магазинам оптики</p> <p>- активный поиск клиентов, создание клиентской базы</p> <p>- контроль дебиторской задолженности</p> <p>- выполнение плана продаж</p> <p>- участие в профильных выставках</p> <p>- частые командировки по России</p> </li> </ul> <strong>Требования:</strong> <ul> <li> <p>- образование: высшее (желательно)</p> <p>-умение вести переговоры‚ проводить личные презентации товара и обучение.</p> <p><strong>-</strong>знание сетевого сегмента оптического рынка, опыт работы в области оптовых продаж</p> <p>- навыки телефонных продаж</p> <p>- знание ПК</p> <p> </p> </li> </ul> <strong>Условия:</strong> <ul> <li> <p>- заработная плата: оклад 40 000 +% за выполнение плана</p> <p>- график работы: понедельник - пятница с 10:00 до 18:00</p> </li> </ul>",
    "area_id": 1,
    "cre

Для этого бейзлана нам понадобятся только хэши работодателей, поэтому будет оставлять только их для экономии памяти:

In [8]:
%%time

vacancy_employers = {int(vacancy_id): vacancy['employer'] for vacancy_id, vacancy in part.items()}

for part_num in tqdm(range(2, 11)):
    part = read_vacancies_part(part_num)
    vacancy_employers.update({int(vacancy_id): vacancy['employer'] for vacancy_id, vacancy in part.items()})
    print(f'Всего вакансий в части {part_num}: {len(part):,d}')

print(f'Всего вакансий в датасете: {len(vacancy_employers):,d}')
del part

Всего вакансий в части 2: 300,000
Всего вакансий в части 3: 300,000
Всего вакансий в части 4: 300,000
Всего вакансий в части 5: 300,000
Всего вакансий в части 6: 300,000
Всего вакансий в части 7: 300,000
Всего вакансий в части 8: 300,000
Всего вакансий в части 9: 300,000
Всего вакансий в части 10: 212,650

Всего вакансий в датасете: 2,912,650
Wall time: 1min 14s


Прочитаем файл со специализациями из тренировочного множества:

In [10]:
train_specializations = pd.read_csv(data_path+'train_labels.csv.gz', compression='gzip')
train_specializations.head(5)

,vacancy_id,specializations
0,1,"[242, 256, 302, 324, 358, 440]"
1,3,[211]
2,4,"[389, 412, 437]"
3,6,[445]
4,9,[503]


Для удобства превратим прочитанный датафрейм в словарь из id вакансии в список id специализаций:

In [11]:
train_specializations = {
    vacancy_id: list(map(int, specs[1:-1].split(',')))
    for vacancy_id, specs in train_specializations.set_index('vacancy_id')['specializations'].iteritems()
}

Наконец, прочитаем файл с id вакансий из тестового множества:

In [13]:
test_ids = pd.read_csv(data_path+'test_vacancy_ids.csv.gz', compression='gzip').values.ravel()

### Подготовка сабмита

Подготовим сабмит, основанный на интуиции о том, что работодатели часто создают вакансии с одними и теми же специализациями. Поэтому для каждой вакансии из тествого множества поставим в соответствие:
* список из top3 специализаций работодателя, если он встречался в тренировочном множесте
* одну самую популярную специализацию во всём тренировочном множестве, если мы не видели этого работодателя

Посчитаем встречаемость каждой специализации в целом по всем вакансиям, а также в отдельности по каждому работодателю:

In [14]:
%%time

common_counter = Counter()
employer_counters = defaultdict(lambda: Counter())

for vacancy_id, specs in tqdm(train_specializations.items()):
    employer_counter = employer_counters[vacancy_employers[vacancy_id]]
    for specialization in specs:
        common_counter[specialization] += 1
        employer_counter[specialization] += 1


Wall time: 4.27 s


Теперь нам известен top3 самых популярных специализаций по каждому работодателю, а также самая популярная специализация в датасете:

In [15]:
top_spec = common_counter.most_common(1)[0][0]
top3_specs_by_employer = {
    employer: list(map(itemgetter(0), counter.most_common(3)))
    for employer, counter in employer_counters.items()
}

print(f'top_spec: {top_spec}')
print('---')
for employer, employer_specs in islice(top3_specs_by_employer.items(), 10):
    print(f'{employer}: {employer_specs}')

top_spec: 256
---
0ce23382345c: [256, 231, 535]
11ecc72a7a76: [221, 117, 82]
e1e424ceb5e4: [389, 412, 437]
6bba39296047: [445, 311, 477]
01a0c3e3c71c: [503]
4f31af482a54: [267, 503, 573]
fe75299b4202: [256, 520, 535]
298d6ba1690e: [16, 235, 81]
2d136aa135b3: [149, 183, 196]
67b00ca54374: [196, 417]


Составим сабмит по описанному выше принципу и запишем его в таком же формате, как и файл со специализациями из тренировочного множества:

In [19]:
submissions_path

'..\\Submissions'

In [18]:
%%time
sample_submission = pd.DataFrame([
    (vacancy_id, top3_specs_by_employer.get(vacancy_employers[vacancy_id], [top_spec]))
    for vacancy_id in test_ids
], columns=['vacancy_id', 'specializations'])
sample_submission.to_csv(submissions_path+f'sample_submission_{"2020-05-25"}_{1}.csv.gz', index=False, compression='gzip')

Wall time: 14 s


Такой сабмит даёт **0.23546** mean f-score на public.